# Numeric case

In [ ]:
%matplotlib inline

import pandas as pd
pd.set_option('display.max_columns', None)
from ollama_models import ollama_models
models = ollama_models()

models

model = models[0]

from community_alignment import community_alignment

df = community_alignment('deepseek-r1:1.5b-qwen-distill-q4_K_M')

from load_saved_questions import load_saved_questions
from call_local_llm import call_local_llm

df.question_type.unique()

df[['id', 'question_type', 'error']].sort_values(by='error', ascending=False)

question = load_saved_questions([3095])[0]

from community_forecast import community_forecast
import pandas as pd
from flatten_dict import flatten_dict
from datetime import datetime
from gather_research_and_set_prompt import gather_research_and_set_prompt
from generate_forecasts_and_update_rag import generate_forecasts_and_update_rag
from tqdm import tqdm
import time

print('START model', model, 'id', id)
start_time = time.time()
questions = [question]
id = question.id_of_question
id_to_forecast = {question.id_of_question: community_forecast(question) for question in questions}
df = pd.DataFrame([flatten_dict(q.api_json, sep='_') for q in questions])
df['id_of_post'] = [q.id_of_post for q in questions]
df['id_of_question'] = [q.id_of_question for q in questions]
df['question_options'] = df['question_options'].apply(repr)
df['today'] = datetime.now().strftime("%Y-%m-%d")
df['crowd'] = df.apply(lambda row: id_to_forecast[row.id_of_question], axis=1)

df1 = df[['id_of_question', 'id_of_post', 'today', 'open_time', 'scheduled_resolve_time', 'title',
    'question_resolution_criteria', 'question_fine_print', 'question_type', 
     'question_description',
    'question_options', 'question_group_variable', 'question_question_weight',
    'question_unit', 'question_open_upper_bound', 'question_open_lower_bound',
    'question_scaling_range_max', 'question_scaling_range_min', 'question_scaling_zero_point','crowd']].copy()

df2, rag = gather_research_and_set_prompt(df1, True, model)

df3 = df2[['id_of_question', 'title',
        'question_resolution_criteria', 'question_fine_print', 'question_type', 
         'question_description',
        'question_options', 'question_group_variable', 'question_question_weight',
        'question_unit', 'question_open_upper_bound', 'question_open_lower_bound',
        'question_scaling_range_max', 'question_scaling_range_min', 'question_scaling_zero_point','crowd',
          'research', 'asknews', 'learning']].copy()

row = df3.iloc[0]

row

for item, value in row.items():
    print(item.upper())
    print()
    print(value)
    print('===============================')
    print()

row.question_resolution_criteria

prompt = f"""Here is some information about a question:

TITLE
=====

{row.title}

RESOLUTION CRITERIA
===================

{row.question_resolution_criteria}

SCALE
=====

From {row.question_scaling_range_min} to {row.question_scaling_range_max}

Using the information, answer the following questions:

a.  What is the minimum value for the answer?
b.  What is the maximum value for the answer?
c.  Is there a formula for the answer given in the above text?
d.  If there is a formula, what needs to go into the formula and how do you estimate it?  Write a python function for the formula, commenting its inputs and outputs for clarity
"""

print(prompt)

terms = call_local_llm(prompt, model)

print(terms)

prompt2 = f"""
You are the intelligence community's best geopolitical, economic and overall news trivia forecaster.  
You are given the following information to make a prediction:

```title
{row.title}
```

```news
{row.asknews}

```research
{row.research}
```

Using the title, news and research, make a forecast according to the formulas you derived in the previous prompt response.
If the Python function has multiple variables, assess each variable individually and then use the function to compute the measure.

The last thing you write is your final answer as this sequence of percentile levels in percent and values as floating point numbers without currency symbols, commas or spelled out numbers like "trillion", just the raw complete number:
"
Percentile 10: XX
Percentile 20: XX
Percentile 40: XX
Percentile 60: XX
Percentile 80: XX
Percentile 90: XX
"
Each line of the final answer MUST START with the word "Percentile".  For example if you have "10: 201" instead of "Percentile 10: 201", that is wrong.

PLEASE REMEMBER THAT THE ANSWER MUST BE IN THE TERMS REQUESTED BY THE PROBLEM.  
A QUICK CORRECTNESS CHECK IS THAT YOUR ANSWER MUST LIE BETWEEN THE MINIMUM AND MAXIMUM VALUES SPECIFIED.  
THE REQUESTED TERMS ARE AS FOLLOWS:

```terms
{terms}
```
"""

print(prompt2)

answer_mistral = call_local_llm(prompt2, model_mistral)

answer_mistral2 = call_local_llm(prompt2, model_mistral)

answer_mistral3 = call_local_llm(prompt2, model_mistral)

from extract_forecast import extract_percentile_numbers

forecasts = [extract_percentile_numbers(x) for x in [answer_mistral, answer_mistral2, answer_mistral3]]

forecasts

from median_dictionaries import median_dictionaries

forecast = median_dictionaries(forecasts)

forecast

prompt = f"""
Summarize the gist pf the rationale or thinking of the following answers from 3 different forecasters to a single problem. 

```forecast 1
{answer_mistral}
```

```forecast 2
{answer_mistral2}
```

```forecast 3
{answer_mistral3}
```

DO NOT REFER TO THE 3 FORECASTERS.  PRESENT THIS AS YOUR OWN THINKING, YOUR OWN RATIONALE.  Use as your final the median forecast which is

```median forecast
{forecast}
```
"""

rationale = call_local_llm(prompt, model_mistral)

print(rationale)

row.forecast = rationale

row.prediction = forecast

row.crowd

from error import error

error(row)